In [1]:
#iniciar spark
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
#conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

:: loading settings :: url = jar:file:/Users/felipeazevedo/GITHUB/grupo-g2-pi-2022/PYSPARK-SPRINT-3/env/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/felipeazevedo/.ivy2/cache
The jars for the packages stored in: /Users/felipeazevedo/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-dc94f548-7c2b-42cc-9e31-3eb8732d7689;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
	found com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
downloading https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.2.2/hadoop-aws-3.2.2.jar ...
	[SUCCESSFUL ] org.apache.hadoop#hadoop-aws;3.2.2!hadoop-aws.jar (463ms)
downloading https://repo1.maven.org/maven2/com/microsoft/azure/spark-mssql-connector_2.12/1.2.0/spark-mssql-connector_2.12-1.2.0.jar ...
	[SUCCESSFUL ] com.microsoft.azure#spark-mssql-connector_2.12;1.2

In [4]:
#df = spark.read.option('delimiter', ';').option('header', 'true').csv('s3a://marise-3cco/vendas_anuais.csv')
df = spark.read.option('delimiter', ';').option('header', 'true').csv('dados-template.csv')

In [5]:
df.show()

+----------+------------+--------------+------+
|id_cliente|total_vendas|numero_pedidos|perfil|
+----------+------------+--------------+------+
|    100001|       80064|             3|     F|
|    100002|       21753|             3|     F|
|    100003|        7458|             2|     M|
|    100004|        4986|             3|     M|
|    100005|       72311|             4|     F|
|    100006|        6943|             2|     F|
|    100007|        4015|             2|     M|
|    100008|        5861|             2|     M|
|    100009|       36463|             2|     F|
|    100010|        4431|             2|     M|
|    100011|       21641|             1|     F|
|    100012|       15792|             2|     F|
|    100013|       28958|             1|     M|
|    100014|       10444|             7|     M|
|    100015|         823|             3|     M|
|    100016|        9021|             2|     M|
|    100017|        5909|             2|     M|
|    100018|       36138|             1|

In [8]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

_df = df.select(
    col('id_cliente').cast(IntegerType()).alias('ID_CLIENTE'),
    col('total_vendas').cast(IntegerType()).alias('TOTAL_VENDAS'),
    col('numero_pedidos').cast(IntegerType()).alias('NUMERO_PEDIDOS'),
    col('perfil').alias('PERFIL')
)

_df.write \
    .mode('overwrite') \
    .csv('vendas_anuais')
   #.csv('s3a://marise-3cco/vendas_anuais')

_df.write \
 .format('com.microsoft.sqlserver.jdbc.spark') \
 .mode('overwrite') \
 .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
 .option('url', 'jdbc:sqlserver://felipe.database.windows.net;databaseName=g2-pi-2022;') \
 .option('dbtable', 'VENDAS_ANUAIS') \
 .option('user', 'adminlocal') \
 .option('password', '123mysql@') \
 .save()